In [ ]:
from os import path
from glob import glob
import gc

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
import umap

import differentiation

%matplotlib widget

In [ ]:
data_folder = '../data/monkey/D_20161011_1445'
sample_rate = 5000
adu2uv = 0.195 # convert data to uV units

In [ ]:
header_ids = {
    'bad_packet' : 0b1000000000000000,
    'one_second' : 0b0100000000000000,
    'analog_smp' : 0b0010000000000000,
    'stimulus_c' : 0b0001000000000000,
    'stimulus_b' : 0b0000100000000000,
    'stimulus_a' : 0b0000010000000000,
}

def load_channels(data_folder):
    channels = {}
    chlen = None
    for f in sorted(glob(path.join(data_folder, '*.i16'))):
        name = 'ch'+path.basename(f)[-6:-4]
        channels[name] = np.memmap(f, dtype='int16')
        if chlen is None:
            chlen = len(channels[name])
        if len(channels[name]) != chlen:
            raise ValueError(
                'Channel length mismatch for channel {0:s}'.format(name)
            )
    return channels

def load_events(data_folder):
    files = glob(path.join(data_folder, '*_Events.u32'))
    if len(files) > 1:
        raise ValueError('Found multiple event files.')
    events = np.memmap(files[0], dtype='uint32').reshape(-1, 3)
    events = pd.DataFrame(data=events, columns=['ID', 'Value', 'Timestamp'])
    events['ID'] = events['ID'].map(
        {
            0 : 'condition',
            1 : 'discrimination',
            2 : 'stimulus',
            3 : 'sampling_status',
            4 : 'one_second_marker',
            5 : 'crc_error'
        }
    )
#     events.Timestamp = events.Timestamp / 5000
    return events

def load_headers(data_folder):
    files = glob(path.join(data_folder, '*_Digi00.u16'))
    if len(files) > 1:
        raise ValueError('Found multiple header files.')
    return pd.DataFrame(np.memmap(files[0], dtype='uint16'))

def expand_header(x):
    marks = {}
    for name, field in header_ids.items():
        if x&field:
            marks[name] = True
        else:
            marks[name] = False
    return pd.Series(marks)

# Load data

In [4]:
events = load_events(data_folder)
data = pd.DataFrame(load_channels(data_folder))
data = data[::5] # downsample simply by subsampling
data.index = range(len(data))
sample_rate = 1000
# headers = load_headers(data_folder)

# Extract stimulation times
A single stimulus comprises of a series of pulses (see below). Interval between pulses is 1, 10, 33 or 100 ms; interstimulus interval is 9 or 10 s.  
Given that, it would be interesting to look at data from -1 s to +1 s from stimulus time.

In [5]:
f, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)
f.canvas.header_visible = False
ax.hist(events[events.ID=='stimulus'].Timestamp.diff().dropna()/5000, range=(0, 12), bins=1200, log=True);
ax.set_xlabel('Inter-stimulus time (s)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# f, ax = plt.subplots(1, 1, figsize=(12, 2), tight_layout=True)
# f.canvas.header_visible = False
# (events[events.ID=='stimulus'].Timestamp.diff()/5000).plot(ax=ax);
# # ax.set_xlim(23500, 24500)
# ax.set_xlabel('Time (s)');

## Extract stimulation burst start times

In [7]:
burst_start = events[events.ID=='stimulus'].Timestamp.diff()/5000 > 8
burst_start = events[events.ID=='stimulus'][burst_start].Timestamp.values.astype(int)
# rescale the index to match new sample_rate
burst_start = (burst_start * sample_rate / 5000).astype(np.uint32)
gc.collect()
burst_start

array([   20005,    30005,    40005, ..., 48970005, 48980005, 48990005],
      dtype=uint32)

## Construct windows around stimulation times

In [8]:
def get_windows(data, stimulus_times, sample_rate, forward, backward):
    '''
    forward, backward: window size around stimulus time in s
    '''
    windows = pd.Series(data=np.nan, index=data.index, name='windows')
    windows[stimulus_times] = np.arange(1, 1+len(stimulus_times))
    if forward > 0:
        windows.ffill(limit=int(forward*sample_rate), inplace=True)
    if backward > 0:
        windows.bfill(limit=int(backward*sample_rate), inplace=True)
    windows = windows.replace(np.nan, 0).astype('uint16')
    gc.collect()
    return windows

# UMAP unperturbed states

In [9]:
# # plt.close(f)
# gc.collect()
# f, ax = plt.subplots(1, 1, figsize=(12, 2), tight_layout=True)
# f.canvas.header_visible = False
# for t in data.index[burst_start][::100]:
#     ax.axvline(t/sample_rate, c='k', lw=0.5)
# ax.plot(data.index[::100]/sample_rate, data.ch00[::100], label='Eye')
# ax.plot(data.index[::100]/sample_rate, data.ch02[::100], label='LFP')
# ax.plot(data.index[::100]/sample_rate, data.ch33[::100], label='Accl')
# ax.legend();

## Get unperturbed windows

In [10]:
# let's get windows that start with the stimulus and end just short of (0.5s) the next stimulus
# Then within a window, we will discard the first 2 seconds of data since that still consists of response to stimulation
windows = get_windows(data, burst_start, sample_rate, 9.5, 0)
windows = windows[windows>0]

# now discard the first 2.5 s post stimulus
windows = windows.groupby(windows).apply(lambda s: s[int(2.5*sample_rate):]).droplevel(0)

## Create state vectors

In [11]:
def mean_lfp(aligned_df, sample_rate, winsize):
    """
    Simply returns the mean absolute value of signal
    Instead, it could first find the envelope and return mean amplitude of that
    """
    if len(aligned_df) < winsize * sample_rate:
        # window overlaps with artifact or window too short, so return nan
        return aligned_df.drop('windows', axis=1).mean()*np.nan
    else:
        return aligned_df.drop('windows', axis=1).abs().mean()
    return

def spectral_state(aligned_df, sample_rate, winsize):
    """
    Returns the spectral state for a block of time
    """
    if len(aligned_df) < winsize * sample_rate:
        # window overlaps with artifact | window too short, so return nan
        aligned_df = pd.DataFrame(data=np.zeros((int(sample_rate*winsize), len(aligned_df.columns)-1)))
        return pd.Series(differentiation.spectral_states(
            sample_rate=sample_rate,
            window_length=winsize,
            data=aligned_df.values[:int(winsize*sample_rate)].T
        )[-1])*np.nan
    else:
        spec = differentiation.spectral_states(
            sample_rate=sample_rate,
            window_length=winsize,
            data=aligned_df.drop('windows', axis=1).values[:int(winsize*sample_rate)].T
        )[-1]
        return pd.Series(spec)

def spectral_differentiation(aligned_df, sample_rate, winsize, state_length):
    if len(aligned_df) < winsize * sample_rate:
        return pd.Series([np.nan]*int((winsize/state_length)*(winsize/state_length-1)/2))
    return pd.Series(
        differentiation.spectral_differentiation(
            aligned_df.drop('windows', axis=1).values[:int(winsize*sample_rate)].T,
            sample_rate=sample_rate, window_length=state_length
        )
    )

In [13]:
func = spectral_state
windowed_data = pd.concat([data.reindex(windows.index), windows], axis=1)
states = windowed_data[['ch{0:02d}'.format(i) for i in range(2, 16)]+['windows']].groupby('windows').apply(
    func, sample_rate=sample_rate, winsize=7
).dropna()
states.shape

(4897, 49014)

In [64]:
reducer = umap.UMAP(
    n_neighbors=500,
    min_dist=0.0005,
)
reducer.fit(states.values)
embedding = reducer.transform(states.values)

## Spectral states cluster according to acceleration / motion and time

In [65]:
f, ax = plt.subplots(1, 1, figsize=(6, 4), constrained_layout=True)
accl = windowed_data.groupby('windows').apply(lambda df: df['ch33'].max()-df['ch33'].min()).loc[states.index]
f.canvas.header_visible = False
ax.set_xlabel('umap 1')
ax.set_ylabel('umap 2')
ax.set_title('{0:.2f} s window, {1:s}'.format(7, func.__name__), fontsize=10)

sc = ax.scatter(
    embedding[:, 0], embedding[:, 1],
    c=accl, cmap=cm.RdBu_r, alpha=0.1, vmax=1000,
)
plt.colorbar(cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=accl.min(),vmax=1000,),
    cmap=cm.RdBu_r
), label='Acceleration amplitude');

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
f, ax = plt.subplots(1, 1, figsize=(6, 4), constrained_layout=True)
accl = windowed_data.groupby('windows').apply(lambda df: df['ch33'].max()-df['ch33'].min()).loc[states.index]
f.canvas.header_visible = False
ax.set_xlabel('umap 1')
ax.set_ylabel('umap 2')
ax.set_title('{0:.2f} s window, {1:s}'.format(7, func.__name__), fontsize=10)

sc = ax.scatter(
    embedding[:, 0], embedding[:, 1],
    c=range(len(embedding)), cmap=cm.copper_r, alpha=0.1,
)
plt.colorbar(cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=0,vmax=len(embedding)*10,),
    cmap=cm.copper_r
), label='Time');

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
f, ax = plt.subplots(1, 1, figsize=(6, 4), constrained_layout=True)
eog = windowed_data.groupby('windows').apply(lambda df: (df.ch00+df.ch01).max()-(df.ch00+df.ch01).min()).loc[states.index]
f.canvas.header_visible = False
ax.set_xlabel('umap 1')
ax.set_ylabel('umap 2')
ax.set_title('{0:.2f} s window, {1:s}'.format(7, func.__name__), fontsize=10)

sc = ax.scatter(
    embedding[:, 0], embedding[:, 1],
    c=eog, cmap=cm.PuOr, alpha=0.1, vmax=3000
)
plt.colorbar(cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=eog.min(),vmax=3000,),
    cmap=cm.PuOr
), label='Eye movement');

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

In [40]:
f, ax = plt.subplots(1, 1, figsize=(12, 2))
eog.plot(kind='hist', ax=ax, range=(0, 2000))
# windowed_data.groupby('windows').apply(lambda df: (df.ch00+df.ch01).max()-(df.ch00+df.ch01).min()).plot(ax=ax)
# windowed_data.groupby('windows').apply(lambda df: df['ch33'].max()-df['ch33'].min()).plot(ax=ax.twinx(), c='k', lw=0.25)

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Mean responses to stimulation